In [1]:
import numpy as np
import pandas as pd
import matplotlib as ply
import seaborn as sns

In [2]:
#pd.read_csv??

train = pd.read_csv('dataset/train.csv_VsW9EGx.zip')
test = pd.read_csv('dataset/test.csv_yAFwdy2.zip',compression='infer')
sample = pd.read_csv('dataset/sample_submission_iA3afxn.csv')

#train['source']='train'
#test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print (train.shape, test.shape, data.shape)

(381109, 12) (127037, 11) (508146, 12)


In [3]:
'''Hypothesis Generation
1. People with driving license will take Car Insurance
2. Customer already having car insurance will take the new insurance
3. Less the Vechile Age more hte chances of taking the insurance
4. Longer the customer association more chances of taking car insurance
'''

'Hypothesis Generation\n1. People with driving license will take Car Insurance\n2. Customer already having car insurance will take the new insurance\n3. Less the Vechile Age more hte chances of taking the insurance\n4. Longer the customer association more chances of taking car insurance\n'

In [4]:
#Unique value
data.apply(lambda x: len(x.unique()))

id                      508146
Gender                       2
Age                         66
Driving_License              2
Region_Code                 53
Previously_Insured           2
Vehicle_Age                  3
Vehicle_Damage               2
Annual_Premium           52062
Policy_Sales_Channel       157
Vintage                    290
Response                     3
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508146 entries, 0 to 508145
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    508146 non-null  int64  
 1   Gender                508146 non-null  object 
 2   Age                   508146 non-null  int64  
 3   Driving_License       508146 non-null  int64  
 4   Region_Code           508146 non-null  float64
 5   Previously_Insured    508146 non-null  int64  
 6   Vehicle_Age           508146 non-null  object 
 7   Vehicle_Damage        508146 non-null  object 
 8   Annual_Premium        508146 non-null  float64
 9   Policy_Sales_Channel  508146 non-null  float64
 10  Vintage               508146 non-null  int64  
 11  Response              381109 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 46.5+ MB


In [6]:
#data describe
data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1.0
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0.0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1.0
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0.0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0.0


In [7]:
#Binning Age, Regional Code, Policy Sales Channel
data['age_bins'] = (pd.cut(x=data['Age'], bins=[0,20, 29, 39, 49,59,69,79,89])).astype(object)
data['Region_Code_bins'] = (pd.cut(x=data['Region_Code'], bins=[0,10,19,29,39,49,59])).astype(object)
data['Policy_Sales_Channel_bins'] = (pd.cut(x=data['Policy_Sales_Channel'], bins=[0,25,49,74,99,124,149,174])).astype(object)
data['Vintage_bins'] = (pd.cut(x=data['Vintage'], bins=[0,50,99,149,199,249,299])).astype(object)


In [8]:
#droping Columns not required
data = data.drop(columns = ['id', 'Age', 'Region_Code','Policy_Sales_Channel', 'Vintage'])

In [9]:
#Filter categorical variables
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
#Exclude ID cols and source:
categorical_columns = [x for x in categorical_columns if x not in ['source']]
#Print frequency of categories
for col in categorical_columns:
    print ('\nFrequency of Categories for varible %s'%col)
    print (data[col].value_counts())


Frequency of Categories for varible Gender
Male      274325
Female    233821
Name: Gender, dtype: int64

Frequency of Categories for varible Vehicle_Age
1-2 Year     267015
< 1 Year     219805
> 2 Years     21326
Name: Vehicle_Age, dtype: int64

Frequency of Categories for varible Vehicle_Damage
Yes    256248
No     251898
Name: Vehicle_Damage, dtype: int64

Frequency of Categories for varible age_bins
(20, 29]    198665
(39, 49]    102686
(29, 39]     72344
(49, 59]     63882
(59, 69]     38767
(69, 79]     22107
(0, 20]       8318
(79, 89]      1377
Name: age_bins, dtype: int64

Frequency of Categories for varible Region_Code_bins
(19.0, 29.0]    183091
(0.0, 10.0]      90682
(39.0, 49.0]     83971
(29.0, 39.0]     72316
(10.0, 19.0]     61102
(49.0, 59.0]     14285
Name: Region_Code_bins, dtype: int64

Frequency of Categories for varible Policy_Sales_Channel_bins
(149, 174]    252537
(99, 124]     113868
(25, 49]      111499
(0, 25]        20537
(49, 74]        5360
(124, 149]     

In [10]:
data.dtypes

Gender                        object
Driving_License                int64
Previously_Insured             int64
Vehicle_Age                   object
Vehicle_Damage                object
Annual_Premium               float64
Response                     float64
age_bins                      object
Region_Code_bins              object
Policy_Sales_Channel_bins     object
Vintage_bins                  object
dtype: object

In [11]:
#Label Encoding
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
data['Response'] = le.fit_transform(data['Response'])
var_mod = ['Vehicle_Damage', 'Vehicle_Age']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [12]:
#One Hot Encoding
data = pd.get_dummies(data, columns=categorical_columns)

In [13]:
train = data.iloc[:381109]
test = data.iloc[381109:]
test =  test.drop(columns='Response')
target = train['Response']

X = train.drop(columns='Response')
y = train['Response']

In [ ]:
test.head()

In [14]:
#Splitting Dataset
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2, random_state = 0)

In [15]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)


In [ ]:
from imblearn.combine import SMOTETomek
from collections import Counter

os=SMOTETomek(sampling_strategy=0.75)
X_train_ns,y_train_ns=os.fit_sample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))


In [ ]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_ns, y_train_ns)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_valid)
y_pred_prob = classifier.predict_proba(X_valid)

In [ ]:
y_pred_prob

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, roc_auc_score
cm = confusion_matrix(y_valid, y_pred)
roc_score = roc_auc_score(y_valid, y_pred)

In [ ]:
cm, roc_score

In [ ]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train_ns, y_train_ns)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_valid)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_valid, y_pred)
roc_score = roc_auc_score(y_valid, y_pred)

In [ ]:
cm, roc_score

In [ ]:
y_test = classifier.predict(test)

In [ ]:
y_test

In [ ]:
sample['Response'] = y_test

In [ ]:
#sample.to_csv('crosssel2.csv', index=False)
sample.to_csv('crosssel4.csv', index=False)

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=y_train)
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 10
bst = lgb.train(param, train_data, num_round)
lgb.cv(param, train_data, num_round, nfold=5)
ypred = bst.predict_proba(test)

In [ ]:
sample['Response'] = y_test
sample.to_csv('crosssel5.csv', index=False)
